In [ ]:
!pip install datasets transformers huggingface_hub transformers[torch] accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from huggingface_hub import login

In [ ]:
login()

In [ ]:
import re
from sklearn.model_selection import train_test_split

In [ ]:
f = open("./train.txt", "r")
text = f.readlines()

In [ ]:
len(text)

603

In [ ]:
def build_text_files(data_text, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_text:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(text,test_size=0.15)


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

print(train)
print(test)

Train dataset length: 512
Test dataset length: 91
['Authority control databases Edit this at Wikidata\n', ' Sethuraman, Mahesh (27 February 2013). "Mumbai \'01 – An underrated classic". ESPNcricinfo. Archived from the original on 14 September 2018. Retrieved 14 September 2018.\n', 'Dravid continued his successful run, scoring seven fifties in the next eight Tests that included fifties in six consecutive innings (three each against Sri Lanka and Australia), becoming only the second Indian to do so after Gundappa Vishwanath.[44][60] By the end of 1997-98 Test season, he had scored 15 fifties in 22 Tests including four scores of nineties but just one hundred.[61]\n', "Dravid started playing cricket at the age of 12 and represented Karnataka at the under-15, the under-17, and the under-19 levels.[29] Former cricketer Keki Tarapore first noticed Dravid's talent while coaching at a summer camp in the Chinnaswamy Stadium.[30] Dravid scored a century for his school team.[citation needed] He al

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
train_path = "train_dataset.txt"
test_path = "test_dataset.txt"

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

In [ ]:
def load_dataset(train_path,test_path,tokenizer):
  train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
  test_dataset = TextDataset(tokenizer=tokenizer,file_path=test_path, block_size=64)
  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
  return train_dataset,test_dataset,data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

In [ ]:
model = AutoModelWithLMHead.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1739: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_total_limit=1,
    save_steps=100,
    overwrite_output_dir=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=10, training_loss=3.7875621795654295, metrics={'train_runtime': 12.8835, 'train_samples_per_second': 43.156, 'train_steps_per_second': 0.776, 'total_flos': 18159796224000.0, 'train_loss': 3.7875621795654295, 'epoch': 2.0})

In [ ]:
trainer.save_model()

In [ ]:
import torch

In [ ]:
input_text = "Rahul Dravid as world cup coach won "
input_ids= tokenizer.encode(input_text, return_tensors='pt').to('cuda')
output= model.generate(input_ids, max_length=200, num_return_sequences=1)
output_text= tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Rahul Dravid as world cup coach won                                                                                                                                                                                              


In [ ]:
input_text = "Rahul Dravid son is "
input_ids= tokenizer.encode(input_text, return_tensors='pt').to('cuda')
output= model.generate(input_ids, max_length=200, num_return_sequences=1)
output_text= tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Rahul Dravid son is  a man who has been a key player in the Indian team.                                                                                                                                                                                 


In [ ]:
model.save_pretrained("gpt2-rahuldravid")
tokenizer.save_pretrained("gpt2-rahuldravid")

('gpt2-rahuldravid/tokenizer_config.json',
 'gpt2-rahuldravid/special_tokens_map.json',
 'gpt2-rahuldravid/vocab.json',
 'gpt2-rahuldravid/merges.txt',
 'gpt2-rahuldravid/added_tokens.json',
 'gpt2-rahuldravid/tokenizer.json')

In [ ]:
def evaluate_model(model, dataset):
  trainer = Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      eval_dataset=test_dataset,
  )
  eval_results = trainer.evaluate()
  return eval_results

eval_results_v1 = evaluate_model(model, test_dataset)

model_gpt2 = AutoModelWithLMHead.from_pretrained("gpt2")
eval_results_v2 = evaluate_model(model_gpt2, test_dataset)

print("Evaluation Results (v1):", eval_results_v1)
print("Evaluation Results (v2):", eval_results_v2)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1739: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Evaluation Results (v1): {'eval_loss': 3.378228187561035, 'eval_model_preparation_time': 0.0027, 'eval_runtime': 0.3459, 'eval_samples_per_second': 176.343, 'eval_steps_per_second': 2.891}
Evaluation Results (v2): {'eval_loss': 4.050170421600342, 'eval_model_preparation_time': 0.0029, 'eval_runtime': 0.275, 'eval_samples_per_second': 221.781, 'eval_steps_per_second': 3.636}


In [ ]:
from huggingface_hub import login, create_repo, Repository
login()
repo_name = "fine-tuned-gpt2-rahuldravid"
from huggingface_hub import HfApi


api = HfApi()

# Create a new repository
username = api.whoami()['name']
full_repo_name = f"{username}/{repo_name}"


api.create_repo(repo_name, private=False)

api.upload_folder(
    folder_path='./gpt2-rahuldravid',  # Path to the folder with your model
    repo_id=full_repo_name,  # Model repository name
    commit_message="GPT-2 Rahul Dravid"
)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/metriccoders/fine-tuned-gpt2-rahuldravid/commit/fdc7dd1ad6b2af89e3c8bdeb37dcfeebcec8550f', commit_message='GPT-2 Rahul Dravid', commit_description='', oid='fdc7dd1ad6b2af89e3c8bdeb37dcfeebcec8550f', pr_url=None, pr_revision=None, pr_num=None)